In [ ]:
import numpy as np
import mido
from mido import MidiFile
import os
import pickle

In [ ]:
def calculate_duration(initial, final, timer):
    temp = 0
    for i in range(initial+1, final+1):
        temp = temp + timer[i]        
    return temp

In [ ]:
def calculate_delay(counter, timer, velocities):
    delay = timer[counter]
    
    if counter == 0:
        return delay
    
    for i in range(counter, -1, -1):
        delay = delay + timer[i]
        if timer[i] > 5 and velocities[i] > 0:
            return delay
    
    return delay


In [ ]:
def extract_midi(path):
    deneme=[]
    
    # Initialize a list to store the sequence of notes, velocity, duration and delay
    #path='midi_files/tscha/ty_dezember.mid'
    sequence  = []
    velocity  = []
    delay     = [] #the time between the note and previous note pressed (velocity!=0)
    note_dur  = []
    
    timer     = [] #keeps track of delay(time) in every note
    velocities= []
    
    # Load the MIDI file
    mid = MidiFile(path)
    # Iterate over the tracks in the MIDI file
    for track in mid.tracks:
        temp={}       #keys are notes, values are (i, velocity)
        counter=0     #count note_on messages
        
        # Iterate over the messages in each track
        for msg in (track):
            #print(msg)
            # Check if the message is a note-on message
            if msg.type == 'note_on':
                #print(msg)
                timer.append(msg.time) #write time of every note_on message
                velocities.append(msg.velocity)

                if msg.velocity != 0:
                    temp[msg.note]=(counter, msg.velocity)   
                elif msg.velocity == 0:
                    if msg.note not in temp:
                        continue
                    sequence.append(msg.note)
                    #if not isinstance(temp[msg.note][1], int):
                      #  temp[msg.note][1] = 0
                    velocity.append(temp[msg.note][1])
                    note_dur.append(calculate_duration(temp[msg.note][0], counter, timer))
                    delay.append(calculate_delay(temp[msg.note][0], timer, velocities))
                    del temp[msg.note]


                counter=counter+1    
    
    return np.array(sequence)#np.column_stack()############[len(sequence),len(velocity),len(note_dur),len(delay)])#
#a=extract_midi('')
#a
#Algortim buyuk ihtimal ile farkli zamanlarda baslayip beraber devam eden notalari algilamiyor! dict kullandigim icin

In [ ]:
def artist_collection(path,artist_names):
    '''
    Input: (list) directories of midi files, Output: (list) notes of all songs of all artists
    -----------------------------------------------------------------------------------------
    '''
    temp={}
    notes_dic={}
    file_name=''
    for count, artist in enumerate(artist_names, start=1):
        file_name=file_name+'_'+artist
        the_path=(path+artist+'/')
        files=[i for i in os.listdir(the_path) if i.endswith(".mid")]

        for k, i in enumerate(files, start=1):
            print(f'=================================== Loading files {k}/{len(files)}, artists: {count}/{len(artist_names)}')
            label=f'{artist}_{k}'
            temp[label] = extract_midi(the_path+i)
      
    with open(f'saved_mido_pickle_files/just_notes-{file_name}.pickle', 'wb') as handle:
        pickle.dump(temp, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
    return temp

In [ ]:
def main(path, artist):
    return artist_collection(path,artist_names)

In [ ]:
path='midi_files/'
artist_names=['bach','schubert','beeth']
#the1_path=path+artist_names

all_artist_names = [i for i in os.listdir(path)]
artist_names = all_artist_names[:17]
artist_names

In [ ]:
loaded_dict=main(path, artist_names)

#### das